# General instructions
Basically, all of this was written to help explore problem 11. To start using it, run the first cell in order to use the functions and so on defined in it in later cells. This cell doesn't do much on its own. Scroll down for some examples/other points.

In [1]:
import math
import copy
def hamming(n): # 11=1011_2 -> w(11) = 3
    ans = 0
    while (n > 0):
        ans += n%2
        n //= 2
    return ans
def v2(n): #returns \nu_2(n), ex v2(24) = 3, v2(5) = 0, etc.
    ans = 0
    while (n%2 == 0):
        ans += 1
        n//= 2
    return ans
def vadd(a : list,b : list) -> list: #vector addition, [2]+[3,4] = [5,4]
    if len(b) > len(a): #a is longer
        a,b = b,a
    b = b + [0]*(len(a)-len(b)) #pads zeroes
    ans = []
    for i in range(len(a)):
        ans.append(a[i]+b[i])
    return ans
def tprint(a):
    for i in a:
        out = ""
        for j in i:
            out += "{:2d}".format(j) + " "
        print(out)
class YT:
    def __init__(self,partition : tuple):
        self.part = partition
        self.validate()
        self.sum = sum(partition)
        self.et = []
        for i in partition: #empty-ish tableaux
            self.et.append([-1,]*i)
    def validate(self): #checks to make sure it is valid partition
        localpart = self.part
        for i in range(len(localpart)-1):
            if localpart[i] < localpart[i+1]:
                raise ValueError("Not a valid partition")
        if localpart[-1] <= 0:
            raise ValueError("Not a valid partition")
    def HL(self):
        localpart = self.part
        self.hlt = copy.deepcopy(self.et)
        curx = []
        for i in range(len(localpart)-1,-1,-1): #HL contribution from column
            curx = vadd(curx,[1]*localpart[i])
            self.hlt[i] = vadd(curx,self.hlt[i])
        for i in range(len(localpart)): #HL contribution from row
            for j in range(localpart[i]):
                self.hlt[i][j] += localpart[i]-j
    def v2HL(self):
        ans = 0
        self.v2list = []
        self.v2t = copy.deepcopy(self.hlt)
        for i in range(len(self.hlt)):
            for j in range(len(self.hlt[i])):
                temp = v2(self.hlt[i][j])
                self.v2t[i][j] = temp
                ans += temp
                self.v2list.append(temp)
        self.v2list.sort()
        return ans
    def numSYT(self): #number of syt
        ans = math.factorial(self.sum)
        for i in range(len(self.part)):
            for j in range(self.part[i]):
                ans //= self.hlt[i][j]
        return ans
    def __str__(self):
        out = ""
        out += "Partition: " + str(self.part)
        return out
    def isSym(self): #returns True if symmetric partition
        curx = []
        for i in range(len(self.part)-1,-1,-1):
            curx = vadd(curx,[1]*self.part[i])
        return tuple(curx) == tuple(self.part)
print("Ready to use!")

Ready to use!


## Some examples
The first example has the tuple ```(5,4,3,2,2,1,1)``` represent the partition that makes up the young tableux. Equivalently, the list ```[5,4,3,2,2,1,1]``` should work identically.

In [2]:
part1 = YT((5,4,3,2,2,1,1))
print("The partition:", part1)
print("Sum of parts:", part1.sum)
print("The partition is symmetric: ", part1.isSym())
tprint(part1.et) #tableux with all cells having value -1
part1.HL() #creates and populates part1.hlt with the hook-lengths
print("Hook Lengths:")
tprint(part1.hlt) #part1.HL() MUST BE RUN before part1.hlt can be accessed.
print("v_2 of product of hook lengths:", part1.v2HL())
print("Number of Standard Young Tableux", part1.numSYT())

The partition: Partition: (5, 4, 3, 2, 2, 1, 1)
Sum of parts: 18
The partition is symmetric:  False
-1 -1 -1 -1 -1 
-1 -1 -1 -1 
-1 -1 -1 
-1 -1 
-1 -1 
-1 
-1 
Hook Lengths:
11  8  5  3  1 
 9  6  3  1 
 7  4  1 
 5  2 
 4  1 
 2 
 1 
v_2 of product of hook lengths: 10
Number of Standard Young Tableux 13366080


## Iterating over all partitions of n
The below code allows you to be able to iterate over all partitions of a given integer. However, a word of caution: due to the way it's structured, you can only use each value once, and you must go through the partitions in order (due to this returning a ```generator```). Additionally, it gives ascending partitions, and in order to use it with the above code, you must reverse it.

In [3]:
def accel_asc(n): #from http://jeromekelleher.net/generating-integer-partitions.html
    a = [0 for i in range(n + 1)]
    k = 1
    y = n - 1
    while k != 0:
        x = a[k - 1] + 1
        k -= 1
        while 2 * x <= y:
            a[k] = x
            y -= x
            k += 1
        l = k + 1
        while x <= y:
            a[k] = x
            a[l] = y
            yield a[:k + 2]
            x += 1
            y -= 1
        a[k] = x + y
        y = x + y - 1
        yield a[:k + 1]

### Two last examples using ```accel_asc```

In [7]:
print("All Young Tableux of 5 with Hook-Lengths:")
numsym = 0
for i in accel_asc(8):
    i.reverse() #easiest way to reverse the partition so it's descending
    a = YT(i)
    print("Current partition:", a)
    a.HL()hangouts
    tprint(a.hlt)
print("The number of symmetric Young Tableux of 5:", numsym)

All Young Tableux of 5 with Hook-Lengths:
Current partition: Partition: [1, 1, 1, 1, 1, 1, 1, 1]
 8 
 7 
 6 
 5 
 4 
 3 
 2 
 1 
Current partition: Partition: [2, 1, 1, 1, 1, 1, 1]
 8  1 
 6 
 5 
 4 
 3 
 2 
 1 
Current partition: Partition: [3, 1, 1, 1, 1, 1]
 8  2  1 
 5 
 4 
 3 
 2 
 1 
Current partition: Partition: [2, 2, 1, 1, 1, 1]
 7  2 
 6  1 
 4 
 3 
 2 
 1 
Current partition: Partition: [4, 1, 1, 1, 1]
 8  3  2  1 
 4 
 3 
 2 
 1 
Current partition: Partition: [3, 2, 1, 1, 1]
 7  3  1 
 5  1 
 3 
 2 
 1 
Current partition: Partition: [5, 1, 1, 1]
 8  4  3  2  1 
 3 
 2 
 1 
Current partition: Partition: [2, 2, 2, 1, 1]
 6  3 
 5  2 
 4  1 
 2 
 1 
Current partition: Partition: [4, 2, 1, 1]
 7  4  2  1 
 4  1 
 2 
 1 
hi
Current partition: Partition: [3, 3, 1, 1]
 6  3  2 
 5  2  1 
 2 
 1 
Current partition: Partition: [6, 1, 1]
 8  5  4  3  2  1 
 2 
 1 
Current partition: Partition: [3, 2, 2, 1]
 6  4  1 
 4  2 
 3  1 
 1 
Current partition: Partition: [5, 2, 1]
 7  5  3  2

[3, 1, 1]
True
[3, 1, 1]


In [49]:
def checkallsym(n):
    goal = n-hamming(n) #v2(n!) = n-hamming(n)
    for i in accel_asc(n):
        i.reverse()
        a = YT(tuple(i))
        if a.isSym():
            a.HL()
            if a.v2HL() == goal:
                print(i, "works!")
for j in range(2,30): #1-40 goes by pretty quickly. 1-89 have been verified to not work.
    print("Checking", j)
    checkallsym(j)

Checking 2
Checking 3
Checking 4
Checking 5
Checking 6
Checking 7
Checking 8
Checking 9
Checking 10
Checking 11
Checking 12
Checking 13
Checking 14
Checking 15
Checking 16
Checking 17
Checking 18
Checking 19
Checking 20
Checking 21
Checking 22
Checking 23
Checking 24
Checking 25
Checking 26
Checking 27
Checking 28
Checking 29


#### Additional Documentation (little use)
```hamming(n)``` gives the hamming weight, $\omega$ of $n$, for example $11=1011_2 \implies \omega(11) = 3$.

```v2(n)``` gives the 2-adic evaluation of $n$ (works for integers only).

```vadd(a,b)``` gives the vector sum $a+b$, and it will pad zeroes onto the end, so for example, $(2,3)+(3,4,5)=(5,7,5)$.

```tprint(a)``` gives a nice way of seeing the tableux represented by ```a```, which is stored as ```list[list[]]```


In [10]:
a = YT((7,5,5,3,1,1))
a.HL()
a.v2HL()
tprint(a.hlt)
print("Removing top right hook")
a = YT((4,4,4,3,1,1))
a.HL()
tprint(a.hlt)
print()
#print(a.v2list)
print("Removing bottom hook")
a = YT((4,4,4))
a.HL()
tprint(a.hlt)
print("Removing non-rim hook")
a = YT((4,3))
a.HL()
tprint(a.hlt)
print("Same")
a = YT((2,))
a.HL()
tprint(a.hlt)
#print()
#b = YT((5,3))
#b.HL()
#b.v2HL()
#tprint(b.hlt)
#print()
#tprint(b.v2t)

12  9  8  6  5  2  1 
 9  6  5  3  2 
 8  5  4  2  1 
 5  2  1 
 2 
 1 
Removing top right hook
 9  6  5  3 
 8  5  4  2 
 7  4  3  1 
 5  2  1 
 2 
 1 

Removing bottom hook
 6  5  4  3 
 5  4  3  2 
 4  3  2  1 
Removing non-rim hook
 5  4  3  1 
 3  2  1 
Same
 2  1 


In [11]:
a = YT((10,7,4,3,3,2,1,1,1,1))
a.HL()
a.v2HL()
tprint(a.hlt)
print("Removing top right hook")
a = YT((4,4,4,3,1,1))
a.HL()
tprint(a.hlt)
print()

19 14 12  9  7  6  5  3  2  1 
15 10  8  5  3  2  1 
11  6  4  1 
 9  4  2 
 8  3  1 
 6  1 
 4 
 3 
 2 
 1 
Removing top right hook
 9  6  5  3 
 8  5  4  2 
 7  4  3  1 
 5  2  1 
 2 
 1 

